In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [134]:
#导入助学金数据（这个表里的id是最全的，所以需要以这个表里id为准）
#score特征已经处理，这里就不处理了，现在只处理card,dorm特征.别library,borrow缺失的全置0
#train
subsidy_df=pd.read_csv('all_feature_input/subsidy_train_feature.csv')
card_df=pd.read_csv('all_feature_input/card_train_feature_f.csv')
dorm_df=pd.read_csv('all_feature_input/dorm_train_feature_f.csv')


#test
#subsidy_df=pd.read_csv('all_feature_input/subsidy_test_feature.csv')
#card_df=pd.read_csv('all_feature_input/card_test_feature_ff.csv')
#dorm_df=pd.read_csv('all_feature_input/dorm_test_feature_f.csv')

In [186]:
#处理card缺失数据

#train
card_df=pd.read_csv('all_feature_input/card_train_feature_f.csv')
card_feature=pd.merge(subsidy_df,card_df,how='left',on='id')

In [6]:
card_feature.head()

,id,money,POS_recharge_mean,amount_mean,bus_mean,canteen_mean,card_recharge_mean,f_ten_canteen_mean,f_ten_place_mean,hospital_mean,...,amount_slice_day_0_10_count,amount_slice_day_10_20_count,amount_slice_day_20_30_count,amount_slice_day_30_40_count,amount_slice_day_40_50_count,amount_slice_day_0_10day_rate,amount_slice_day_10_20day_rate,amount_slice_day_20_30day_rate,amount_slice_day_30_40day_rate,amount_slice_day_40_50day_rate
0,0,0,0.949264,7.830016,0.162029,0.206219,0.036007,0.062193,0.409165,0.0,...,82.0,52.0,20.0,8.0,2.0,0.440860,0.279570,0.107527,0.043011,0.010753
1,1,0,0.968610,8.784709,0.086697,0.387145,0.016442,0.046338,0.162930,0.0,...,29.0,38.0,32.0,19.0,2.0,0.207143,0.271429,0.228571,0.135714,0.014286
2,8,0,0.970428,8.388444,0.197665,0.048249,0.028016,0.048249,0.759533,0.0,...,15.0,130.0,84.0,14.0,2.0,0.053381,0.462633,0.298932,0.049822,0.007117
3,9,0,0.973231,9.033875,0.019758,0.140217,0.026769,0.140217,0.670491,0.0,...,16.0,67.0,83.0,26.0,24.0,0.059041,0.247232,0.306273,0.095941,0.088561
4,10,0,0.957882,7.191348,0.145608,0.239471,0.031288,0.095066,0.308063,0.0,...,91.0,56.0,24.0,4.0,4.0,0.435407,0.267943,0.114833,0.019139,0.019139


In [72]:
#card特征统计量

card_sts=card_df['card_day_sum'].describe()
#四分位距
card_IQR=card_sts.ix['75%']-card_sts.ix['25%']
#小于（下四分位数-1.5*四分位距）或大于（上四分位数+1.5*四分位距）的值可以认为是异常值

#这里只认为“消费总天数小于137”的为异常值
card_outlier=card_sts['25%']-1.5*card_IQR
card_outlier

137.0

In [194]:
def card_preprocessing(df):
    #特征名
    a=[x for x in df.columns if x not in ['id','money']]
    #异常数据
    df.fillna(value=0,inplace=True)
    df_outlier=df[df['card_day_sum']<=137]
    #正常数据
    df_normal=df[df['card_day_sum']>137]
    #迭代为异常id的每个特征填充随机数（每个特征均值一个标准差范围内）
    for i in range(len(a)):
        df_mean=df_normal[a[i]].mean()
        df_std=df_normal[a[i]].std()
        rand_card=np.random.uniform(df_mean - df_std, df_mean + df_std, size = len(df_outlier))
        df_outlier[a[i]]=rand_card
    df=pd.concat([df_outlier,df_normal])
    return df

#运用函数
card_feature=card_preprocessing(card_feature)

In [195]:
card_feature['amount_mean'].mean()-card_feature['amount_mean'].std()

5.5452497163356549

In [198]:
np.random.uniform(5.545,6.645,size=100)

array([ 6.07116363,  5.58225091,  5.59399769,  6.61617361,  5.57938097,
        6.52594947,  5.8227739 ,  6.56570521,  5.64702263,  6.57668635,
        6.39266882,  6.11393846,  6.25157815,  6.59858277,  6.37361957,
        6.06940796,  5.80473594,  5.65444529,  5.7972087 ,  6.38970384,
        6.59631583,  5.75567432,  5.61619622,  6.06737115,  5.69026465,
        6.42170222,  6.60520626,  6.54069164,  5.9724072 ,  5.6844214 ,
        6.20325149,  6.27297864,  6.08324327,  6.54114539,  6.57844968,
        5.94175568,  6.61968736,  5.78288673,  5.67067751,  6.22024152,
        5.74615422,  6.29745714,  5.72444391,  5.55620904,  6.30153809,
        6.50409906,  5.8445152 ,  6.20496255,  6.51400852,  5.98356049,
        6.4047791 ,  5.96815424,  5.663256  ,  6.29197106,  5.91858679,
        6.22327351,  6.45702858,  6.01842108,  6.62523201,  6.39702842,
        5.65868678,  6.14886235,  6.36033666,  6.37205508,  6.38449085,
        6.35713756,  6.34881739,  5.83208379,  5.66767339,  5.87

In [199]:
card_feature['amount_mean'].describe()

count    10860.000000
mean         7.340235
std          1.794985
min          1.705851
25%          6.117999
50%          7.226439
75%          8.427290
max         21.434464
Name: amount_mean, dtype: float64

In [208]:
card_feature[card_feature['amount_mean']>10]

,id,money,POS_recharge_mean,amount_mean,bus_mean,canteen_mean,card_recharge_mean,f_ten_canteen_mean,f_ten_place_mean,hospital_mean,...,amount_slice_day_0_10_count,amount_slice_day_10_20_count,amount_slice_day_20_30_count,amount_slice_day_30_40_count,amount_slice_day_40_50_count,amount_slice_day_0_10day_rate,amount_slice_day_10_20day_rate,amount_slice_day_20_30day_rate,amount_slice_day_30_40day_rate,amount_slice_day_40_50day_rate
9,22,1000,0.978818,11.550268,0.053512,0.593088,0.016722,0.045708,0.323300,0.000000,...,56.0,134.0,57.0,9.0,9.0,0.197880,0.473498,0.201413,0.031802,0.031802
10,23,1500,0.972892,10.378614,0.133534,0.489960,0.018072,0.115462,0.371486,0.000000,...,48.0,75.0,86.0,30.0,9.0,0.179775,0.280899,0.322097,0.112360,0.033708
27,76,0,0.960428,11.425765,0.044920,0.173262,0.000000,0.040642,0.567914,0.000000,...,46.0,92.0,73.0,21.0,10.0,0.173585,0.347170,0.275472,0.079245,0.037736
28,77,0,0.939683,16.190492,0.023810,0.112698,0.000000,0.041270,0.552381,0.000000,...,63.0,91.0,67.0,23.0,7.0,0.236842,0.342105,0.251880,0.086466,0.026316
55,172,0,0.960503,11.191831,0.274686,0.062837,0.034111,0.062837,0.773788,0.000000,...,44.0,37.0,22.0,22.0,11.0,0.278481,0.234177,0.139241,0.139241,0.069620
57,178,0,0.957380,10.298272,0.206788,0.127861,0.030781,0.127861,0.565904,0.000789,...,46.0,31.0,38.0,33.0,26.0,0.192469,0.129707,0.158996,0.138075,0.108787
66,195,0,0.958115,10.735864,0.218150,0.075044,0.000000,0.073298,0.675393,0.000000,...,81.0,75.0,24.0,11.0,7.0,0.369863,0.342466,0.109589,0.050228,0.031963
67,198,0,0.928270,11.720992,0.240506,0.118143,0.044304,0.075949,0.508439,0.037975,...,73.0,35.0,24.0,15.0,5.0,0.414773,0.198864,0.136364,0.085227,0.028409
95,282,0,0.910936,13.696555,0.081860,0.362148,0.008513,0.093648,0.360838,0.000000,...,34.0,54.0,67.0,30.0,28.0,0.112211,0.178218,0.221122,0.099010,0.092409
96,283,0,0.892442,13.458895,0.066860,0.395349,0.005814,0.127907,0.308140,0.000000,...,74.0,39.0,43.0,11.0,11.0,0.328889,0.173333,0.191111,0.048889,0.048889


In [205]:
card_feature.ix[2911]['card_day_sum']

227.0

In [121]:
card_feature.head()

,id,money,POS_recharge_mean,amount_mean,bus_mean,canteen_mean,card_recharge_mean,f_ten_canteen_mean,f_ten_place_mean,hospital_mean,...,amount_slice_day_0_10_count,amount_slice_day_10_20_count,amount_slice_day_20_30_count,amount_slice_day_30_40_count,amount_slice_day_40_50_count,amount_slice_day_0_10day_rate,amount_slice_day_10_20day_rate,amount_slice_day_20_30day_rate,amount_slice_day_30_40day_rate,amount_slice_day_40_50day_rate
6,19,0,0.957545,6.834162,0.119500,0.353965,0.018161,0.120949,0.419271,0.005129,...,66.745163,110.163563,64.412813,10.177701,9.501978,0.227920,0.431873,0.253690,0.092008,0.035024
8,21,0,0.976598,5.567005,0.060270,0.312263,0.011805,0.059195,0.444359,0.000320,...,88.973856,90.020518,72.083603,14.414939,1.308247,0.220022,0.415115,0.170858,0.084922,0.038611
15,56,0,0.978813,8.681634,0.059471,0.259388,0.021001,0.074608,0.409059,-0.002005,...,94.170195,123.063138,32.119360,24.977552,6.220657,0.383182,0.316745,0.144492,0.090128,0.036781
16,57,0,0.954363,8.703518,0.127044,0.382102,0.021660,0.086100,0.361134,-0.003239,...,68.108522,113.346370,61.538819,9.907952,8.279787,0.206490,0.375507,0.118083,0.092149,0.017694
17,62,0,0.978306,9.091093,0.043256,0.389552,0.006568,0.132421,0.301417,0.003007,...,43.705073,106.157092,32.404811,19.024860,3.344228,0.346294,0.272679,0.115537,0.023097,0.012998


In [135]:
dorm_df.head()

,id,dorm_count_sum,dorm_daycount,dorm_hour00,dorm_hour01,dorm_hour02,dorm_hour03,dorm_hour04,dorm_hour05,dorm_hour06,...,dorm_hour_01_05_day_mean,dorm_hour_afternoon_day_mean,dorm_hour_evening_day_mean,dorm_hour_midnight_day_mean,dorm_hour_morning_day_mean,dorm_on_school_day_mean,dorm_resting_day_day_mean,dorm_vacation_day_mean,dorm_working_day_day_mean,dorm_day_mean
0,0,186,58,1,0,0,0,0,0,0,...,0,1,1,0,0,2,0,0,2,3.206897
1,1,312,97,0,0,0,0,0,0,2,...,0,1,0,0,0,3,0,0,2,3.216495
2,10,135,56,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,2,2.410714
3,19,266,99,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,2,2.686869
4,20,13,9,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1.444444


In [145]:
dorm_sts=dorm_df['dorm_daycount'].describe()

In [146]:
dorm_IQR=dorm_sts.ix['75%']-dorm_sts.ix['25%']
dorm_outlier=dorm_sts.ix['25%']-1.5*dorm_IQR
dorm_outlier

25.0

In [147]:
dorm_sts

count    7834.000000
mean       81.116416
std        27.710348
min         1.000000
25%        70.000000
50%        86.000000
75%       100.000000
max       136.000000
Name: dorm_daycount, dtype: float64

In [148]:
dorm_IQR

30.0

In [149]:
len(dorm_df[dorm_df['dorm_daycount']<25])

482

In [172]:
dorm_feature=pd.merge(subsidy_df,dorm_df,how='left',on='id')

In [180]:
dorm_feature.fillna(value=0,inplace=True)
a=dorm_feature[dorm_feature['dorm_daycount']<25]
len(a)

3533

In [206]:
a[a['money']==1000]

,id,money,dorm_count_sum,dorm_daycount,dorm_hour00,dorm_hour01,dorm_hour02,dorm_hour03,dorm_hour04,dorm_hour05,...,dorm_hour_01_05_day_mean,dorm_hour_afternoon_day_mean,dorm_hour_evening_day_mean,dorm_hour_midnight_day_mean,dorm_hour_morning_day_mean,dorm_on_school_day_mean,dorm_resting_day_day_mean,dorm_vacation_day_mean,dorm_working_day_day_mean,dorm_day_mean
41,138,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
52,161,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
88,245,1000,17.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.700000
123,398,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
124,399,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
174,565,1000,11.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.571429
194,617,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
486,1507,1000,25.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.923077
514,1585,1000,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,2.000000
692,2125,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [181]:
dorm_feature.head()

,id,money,dorm_count_sum,dorm_daycount,dorm_hour00,dorm_hour01,dorm_hour02,dorm_hour03,dorm_hour04,dorm_hour05,...,dorm_hour_01_05_day_mean,dorm_hour_afternoon_day_mean,dorm_hour_evening_day_mean,dorm_hour_midnight_day_mean,dorm_hour_morning_day_mean,dorm_on_school_day_mean,dorm_resting_day_day_mean,dorm_vacation_day_mean,dorm_working_day_day_mean,dorm_day_mean
0,0,0,186.0,58.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,3.206897
1,1,0,312.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.216495
2,8,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,10,0,135.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,2.410714
